In [1]:
import os


In [2]:
%pwd

'/Users/richcriticism/Documents/Documents/Bhavya_code/machine learning/CorrectPose2/research'

In [3]:
os.chdir("/Users/richcriticism/Documents/Documents/Bhavya_code/machine learning/CorrectPose2")

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_model_path: Path
    training_data: Path
    target_data: Path
    params_epochs: int
    params_batch_size: int



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_model
        params = self.params
        training_data = os.path.join(self.config.data_extraction.extracted_data, "training_data.csv")
        create_directories([
            Path(training.root_dir)
        ])
        target_data = os.path.join(self.config.data_extraction.extracted_data, "target_data.csv")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_model_path=Path(prepare_base_model.updated_model_path),
            training_data=Path(training_data),
            target_data=Path(target_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
        )

        return training_config

In [7]:
import time

In [8]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_model_path
        )

    def csv_data(self, training_data: Path, target_data: Path):
        X = pd.read_csv(training_data)
        Y = pd.read_csv(target_data)

        return X,Y

    
    def train_data_generator(self, X, Y, params_batch_size):
        num_samples = len(X)
        while True:

            indices = np.random.permutation(num_samples)
            for i in range(0, num_samples, params_batch_size):

                if i+ params_batch_size > num_samples:
                    break
                batch_indices = indices[i:i+params_batch_size]
                batch_x = X.iloc[batch_indices]
                batch_y = Y.iloc[batch_indices]
                print(np.array(batch_x).shape)
                yield np.array(batch_x), np.array(batch_y)

    
    def validation_data_generator(self, X, Y, params_batch_size):

        x_train, x_test, y_train, y_test = train_test_split(X, Y , test_size=0.25, random_state=42)

        num_samples = len(X)
        while True:

            indices = np.random.permutation(num_samples)
            for i in range(0, num_samples, params_batch_size):

                if i+ params_batch_size > num_samples:
                    break
                batch_indices = indices[i:i+params_batch_size]
                batch_x = X.iloc[batch_indices]
                batch_y = Y.iloc[batch_indices]
                yield np.array(batch_x), np.array(batch_y)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):

        if self.model is None:
            raise ValueError("Base model is not loaded.")

        X, Y = self.csv_data(self.config.training_data, self.config.target_data)
        train_generator = self.train_data_generator(X, Y, self.config.params_batch_size)
        validation_data = self.validation_data_generator(X, Y, self.config.params_batch_size)


        self.model.compile(
            optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
            loss = tf.keras.losses.binary_crossentropy,
            metrics = ['accuracy']
        )

        self.model.fit(
            train_generator,
            epochs=self.config.params_epochs,
            validation_data=validation_data,
            callbacks=callback_list

        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    prepare_callbacks_config = config.get_prepare_callback_config()

    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training = Training(config=training_config)
    training.get_base_model()
    training.train(callback_list=callback_list)

except Exception as e:
    print("An error occurred:", str(e))
    raise e

[2024-04-09 20:19:37,498: INFO : common : yaml file : config/config.yaml loaded successfully]
[2024-04-09 20:19:37,500: INFO : common : yaml file : params.yaml loaded successfully]
[2024-04-09 20:19:37,500: INFO : common : creating directory at path: artifacts]
[2024-04-09 20:19:37,501: INFO : common : creating directory at path: artifacts/training]
[2024-04-09 20:19:37,501: INFO : common : creating directory at path: artifacts/training]
[2024-04-09 20:19:37,502: INFO : common : creating directory at path: artifacts/prepare_callbacks/checkpoint_dir]
[2024-04-09 20:19:37,502: INFO : common : creating directory at path: artifacts/prepare_callbacks/tensorboard_log_dir]
[2024-04-09 20:19:37,576: WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
(100, 6)
(100, 6)
Epoch 1/100
      1/Unknown 1s 945ms/step - accuracy: 0.0100 - loss: 59.4397(100, 6)
(100, 6)
(100, 6)
(

KeyboardInterrupt: 